In [1]:
import torch
import yaml
import ast
#from dig.threedgraph.dataset import QM93D
from dig.threedgraph.dataset import MD17
from dig.threedgraph.dataset.PygTobermorite import Tobermorite
from dig.threedgraph.method import SphereNet #SchNet, DimeNetPP, ComENet
from dig.threedgraph.method import run
from dig.threedgraph.evaluation import ThreeDEvaluator

# Load configs

In [2]:
with open('config.yaml', 'r') as c:
    config = yaml.safe_load(c)
    
# For strings that yaml doesn't parse (e.g. None)
for key, val in config.items():
    if type(val) is str:
        try:
            config[key] = ast.literal_eval(val)
        except (ValueError, SyntaxError):
            pass

In [3]:
name = config['name']
n_train = config['n_train']
n_val = config['n_val']
seed = config['seed']
energy_and_force = config['energy_and_force']
cutoff = config['cutoff']
num_layers = config['num_layers']
hidden_channels = config['hidden_channels']
out_channels = config['out_channels']
int_emb_size = config['int_emb_size']
basis_emb_size_dist = config['basis_emb_size_dist']
basis_emb_size_angle = config['basis_emb_size_angle']
basis_emb_size_torsion = config['basis_emb_size_torsion']
out_emb_channels = config['out_emb_channels']
num_spherical = config['num_spherical']
num_radial = config['num_radial']
envelope_exponent = config['envelope_exponent']
num_before_skip = config['num_before_skip']
num_after_skip = config['num_after_skip']
num_output_layers = config['num_output_layers']

epochs = config['epochs']
batch_size = config['batch_size']
vt_batch_size = config['vt_batch_size']
lr = config['lr']
lr_decay_factor = config['lr_decay_factor']
lr_decay_step_size = config['lr_decay_step_size']

In [4]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda', index=0)

# Dataset

In [5]:
dataset = Tobermorite(root='dataset/', name=name)

split_idx = dataset.get_idx_split(len(dataset.data.y), train_size=n_train, valid_size=n_val, seed=seed)

train_dataset, valid_dataset, test_dataset = dataset[split_idx['train']], dataset[split_idx['valid']], dataset[split_idx['test']]
print('train, validaion, test:', len(train_dataset), len(valid_dataset), len(test_dataset))

train, validaion, test: 950 50 6000


/home/zkm/anaconda3/envs/spherenet/lib/python3.10/site-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


# Model

In [6]:
model = SphereNet(energy_and_force=energy_and_force, cutoff=cutoff, num_layers=num_layers, 
        hidden_channels=hidden_channels, out_channels=out_channels, int_emb_size=int_emb_size, 
        basis_emb_size_dist=basis_emb_size_dist, basis_emb_size_angle=basis_emb_size_angle, 
        basis_emb_size_torsion=basis_emb_size_torsion, out_emb_channels=out_emb_channels, 
        num_spherical=num_spherical, num_radial=num_radial, envelope_exponent=envelope_exponent, 
        num_before_skip=num_before_skip, num_after_skip=num_after_skip, num_output_layers=num_output_layers 
        )
loss_func = torch.nn.L1Loss()
evaluation = ThreeDEvaluator()

# FLOPs

In [7]:
from torch_geometric.data import DataLoader
from tqdm import tqdm
from thop import profile

train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
for step, batch_data in enumerate(tqdm(train_loader)):
    print(device)
    batch_data = batch_data.to(device)
    inputs = batch_data
    model = model.to(device)
    flops,params=profile(model,(inputs, ))
    print(flops,params)

/home/zkm/anaconda3/envs/spherenet/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
  0%|                                                                                                                                             | 0/238 [00:00<?, ?it/s]

cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.


  0%|▌                                                                                                                                    | 1/238 [00:04<17:18,  4.38s/it]

2602690688.0 1877952.0
cuda:0


  1%|█                                                                                                                                    | 2/238 [00:04<07:53,  2.01s/it]

...

[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
2646949888.0 1877952.0
cuda:0


 60%|██████████████████████████████████████████████████████████████████████████████▏                                                    | 142/238 [00:51<00:32,  2.95it/s]

[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
2604437504.0 1877952.0
cuda:0


 60%|██████████████████████████████████████████████████████████████████████████████▏                                                    | 142/238 [00:51<00:34,  2.76it/s]


KeyboardInterrupt: 